<a href="https://colab.research.google.com/github/navido89/DS-Job-Market-Analysis/blob/master/prioritization_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [287]:
import pandas as pd

In [288]:
# import CCVI & census data csv
df_raw = pd.read_csv('https://storage.googleapis.com/ccvi_county/ccvi_county.csv') #, index_col='[index]')
df_raw.head()

,State,State Abbreviation,County,FIPS (5-digit),THEME 1: Socioeconomic Status,THEME 2:\nHousehold Composition & Disability,THEME 3: \nMinority Status & Language,THEME 4: \nHousing Type & Transportation,THEME 5: Epidemiological Factors,THEME 6: Healthcare System Factors,CCVI SCORE\nHigher = More Vulnerable
0,Alabama,AL,Autauga,1001,0.357,0.566,0.592,0.314,0.833,0.935,0.777
1,Alabama,AL,Baldwin,1003,0.240,0.244,0.432,0.335,0.380,0.768,0.362
2,Alabama,AL,Barbour,1005,0.973,0.839,0.897,0.640,0.778,0.936,0.991
3,Alabama,AL,Bibb,1007,0.746,0.249,0.443,0.743,0.726,0.927,0.775
4,Alabama,AL,Blount,1009,0.635,0.432,0.583,0.322,0.630,0.509,0.612


In [289]:
# filter out California data

# surpress SettingWithCopyError message 
pd.options.mode.chained_assignment = None

ca_df_raw = df_raw.loc[df_raw['State Abbreviation'] == "CA"]
ca_df_raw.head()

,State,State Abbreviation,County,FIPS (5-digit),THEME 1: Socioeconomic Status,THEME 2:\nHousehold Composition & Disability,THEME 3: \nMinority Status & Language,THEME 4: \nHousing Type & Transportation,THEME 5: Epidemiological Factors,THEME 6: Healthcare System Factors,CCVI SCORE\nHigher = More Vulnerable
186,California,CA,Alameda,6001,0.184,0.028,0.971,0.604,0.093,0.022,0.097
187,California,CA,Alpine,6003,0.813,0.787,0.747,0.989,0.288,0.003,0.515
188,California,CA,Amador,6005,0.387,0.303,0.612,0.593,0.396,0.006,0.156
189,California,CA,Butte,6007,0.607,0.327,0.776,0.758,0.057,0.111,0.235
190,California,CA,Calaveras,6009,0.362,0.639,0.548,0.126,0.230,0.032,0.098


In [290]:
# check if this is has the correct number of counties. California has 58 counties
len(ca_df_raw)

58

In [291]:
# add covid information for each county. CC stands for confirmed cases
df_covid_cc_US = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")

df_covid_cc_CA = df_covid_cc_US.loc[df_covid_cc_US['Province_State'] == "California"]
# check if we have 58 counties
#len(df_covid_cc_CA)
#60

df_covid_cc_CA.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,...,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
195,84006001,US,USA,840,6001.0,Alameda,California,US,37.646294,-121.892927,"Alameda, California, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,49796,50405,50946,51590,52475,53518,54518,55073,55899,56432,57081,57921,59172,60125,61111,62046,62943,63866,64760,65679,68016,67375,67952,68649,69107,69554,69693,70334,70823,71298,72024,72597,73115,73542,73771,74087,74661,74959,75175,78224
196,84006003,US,USA,840,6003.0,Alpine,California,US,38.596786,-119.822359,"Alpine, California, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,65,66,67,67,67,68,68,71,71,71,71,71,72,72,72,73,73,73,73,73,73,73,73,73,73,73,73,73,73,74,74,74,74,74,74,74,76,78,78,72
197,84006005,US,USA,840,6005.0,Amador,California,US,38.445831,-120.656960,"Amador, California, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2394,2442,2502,2515,2562,2613,2671,2701,2717,2738,2761,2761,2811,2824,2861,2867,2897,2921,2951,2965,2986,3001,3016,3024,3036,3038,3042,3049,3067,3074,3113,3116,3147,3156,3199,3227,3234,3275,3281,3288
198,84006007,US,USA,840,6007.0,Butte,California,US,39.667278,-121.600525,"Butte, California, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7051,7173,7269,7390,7405,7589,7750,7850,7963,8096,8217,8278,8430,8552,8665,8795,8891,9013,9127,9233,9359,9411,9474,9528,9582,9643,9711,9757,9820,9865,9943,9988,10037,10122,10177,10215,10263,10307,10355,10386
199,84006009,US,USA,840,6009.0,Calaveras,California,US,38.205371,-120.552913,"Calaveras, California, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1097,1102,1111,1131,1161,1186,1230,1248,1276,1292,1321,1342,1385,1425,1449,1460,1475,1493,1513,1538,1596,1608,1617,1621,1637,1647,1672,1695,1704,1713,1723,1737,1747,1757,1763,1778,1785,1786,1799,1808


In [292]:
# need to clean the data and find the 2 extra records in df_covid_cc_CA

# first grab all the county names in combined_key and remove the California, US to get the County names.
df_covid_cc_CA['Combined_Key'] = df_covid_cc_CA['Combined_Key'].apply(lambda x: x.split(",")[0])
df_covid_cc_CA.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,...,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
195,84006001,US,USA,840,6001.0,Alameda,California,US,37.646294,-121.892927,Alameda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,49796,50405,50946,51590,52475,53518,54518,55073,55899,56432,57081,57921,59172,60125,61111,62046,62943,63866,64760,65679,68016,67375,67952,68649,69107,69554,69693,70334,70823,71298,72024,72597,73115,73542,73771,74087,74661,74959,75175,78224
196,84006003,US,USA,840,6003.0,Alpine,California,US,38.596786,-119.822359,Alpine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,65,66,67,67,67,68,68,71,71,71,71,71,72,72,72,73,73,73,73,73,73,73,73,73,73,73,73,73,73,74,74,74,74,74,74,74,76,78,78,72
197,84006005,US,USA,840,6005.0,Amador,California,US,38.445831,-120.656960,Amador,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2394,2442,2502,2515,2562,2613,2671,2701,2717,2738,2761,2761,2811,2824,2861,2867,2897,2921,2951,2965,2986,3001,3016,3024,3036,3038,3042,3049,3067,3074,3113,3116,3147,3156,3199,3227,3234,3275,3281,3288
198,84006007,US,USA,840,6007.0,Butte,California,US,39.667278,-121.600525,Butte,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7051,7173,7269,7390,7405,7589,7750,7850,7963,8096,8217,8278,8430,8552,8665,8795,8891,9013,9127,9233,9359,9411,9474,9528,9582,9643,9711,9757,9820,9865,9943,9988,10037,10122,10177,10215,10263,10307,10355,10386
199,84006009,US,USA,840,6009.0,Calaveras,California,US,38.205371,-120.552913,Calaveras,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1097,1102,1111,1131,1161,1186,1230,1248,1276,1292,1321,1342,1385,1425,1449,1460,1475,1493,1513,1538,1596,1608,1617,1621,1637,1647,1672,1695,1704,1713,1723,1737,1747,1757,1763,1778,1785,1786,1799,1808


In [293]:
print(len(df_covid_cc_CA.Combined_Key.values))
print(len(ca_df_raw.County.values))

60
58


In [294]:
lst1 = df_covid_cc_CA.Combined_Key.values
lst2 = ca_df_raw.County.values

new_list = [
    list(set(lst1).difference(lst2))
]
new_list

# We see covid confirmed cases US has two rows that we have to remove 'Out of CA', 'Unassigned'

[['Unassigned', 'Out of CA']]

In [295]:
# extract rows 'Out of CA', 'Unassigned'
df_covid_cc_CA.drop(df_covid_cc_CA.loc[df_covid_cc_CA['Combined_Key']=="Out of CA"].index, inplace=True)
df_covid_cc_CA.drop(df_covid_cc_CA.loc[df_covid_cc_CA['Combined_Key']=="Unassigned"].index, inplace=True)
len(df_covid_cc_CA)

58

In [296]:
# Check if we have identical Counties
import numpy as np
check_counties = (np.array(ca_df_raw.County.values) == np.array(df_covid_cc_CA.Combined_Key.values))
check_counties

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [297]:
# creata total_cases row
df_covid_cc_CA['total_cases']= df_covid_cc_CA[df_covid_cc_CA.columns[-1]]
df_covid_cc_CA.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,...,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,total_cases
195,84006001,US,USA,840,6001.0,Alameda,California,US,37.646294,-121.892927,Alameda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,50405,50946,51590,52475,53518,54518,55073,55899,56432,57081,57921,59172,60125,61111,62046,62943,63866,64760,65679,68016,67375,67952,68649,69107,69554,69693,70334,70823,71298,72024,72597,73115,73542,73771,74087,74661,74959,75175,78224,78224
196,84006003,US,USA,840,6003.0,Alpine,California,US,38.596786,-119.822359,Alpine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,66,67,67,67,68,68,71,71,71,71,71,72,72,72,73,73,73,73,73,73,73,73,73,73,73,73,73,73,74,74,74,74,74,74,74,76,78,78,72,72
197,84006005,US,USA,840,6005.0,Amador,California,US,38.445831,-120.656960,Amador,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2442,2502,2515,2562,2613,2671,2701,2717,2738,2761,2761,2811,2824,2861,2867,2897,2921,2951,2965,2986,3001,3016,3024,3036,3038,3042,3049,3067,3074,3113,3116,3147,3156,3199,3227,3234,3275,3281,3288,3288
198,84006007,US,USA,840,6007.0,Butte,California,US,39.667278,-121.600525,Butte,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7173,7269,7390,7405,7589,7750,7850,7963,8096,8217,8278,8430,8552,8665,8795,8891,9013,9127,9233,9359,9411,9474,9528,9582,9643,9711,9757,9820,9865,9943,9988,10037,10122,10177,10215,10263,10307,10355,10386,10386
199,84006009,US,USA,840,6009.0,Calaveras,California,US,38.205371,-120.552913,Calaveras,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1102,1111,1131,1161,1186,1230,1248,1276,1292,1321,1342,1385,1425,1449,1460,1475,1493,1513,1538,1596,1608,1617,1621,1637,1647,1672,1695,1704,1713,1723,1737,1747,1757,1763,1778,1785,1786,1799,1808,1808


In [298]:
# Check total confirmed total cases in CA
df_covid_cc_CA["total_cases"].sum()

3399745

In [299]:
# Now merge Lat, Lon_ and total cases with ca_df_raw
ca_df_raw["Longitude"] = df_covid_cc_CA.Long_.values
ca_df_raw["Latitude"] = df_covid_cc_CA.Lat.values
ca_df_raw["Total Covid19 Cases"] = df_covid_cc_CA.total_cases.values

ca_df_raw.head()

,State,State Abbreviation,County,FIPS (5-digit),THEME 1: Socioeconomic Status,THEME 2:\nHousehold Composition & Disability,THEME 3: \nMinority Status & Language,THEME 4: \nHousing Type & Transportation,THEME 5: Epidemiological Factors,THEME 6: Healthcare System Factors,CCVI SCORE\nHigher = More Vulnerable,Longitude,Latitude,Total Covid19 Cases
186,California,CA,Alameda,6001,0.184,0.028,0.971,0.604,0.093,0.022,0.097,-121.892927,37.646294,78224
187,California,CA,Alpine,6003,0.813,0.787,0.747,0.989,0.288,0.003,0.515,-119.822359,38.596786,72
188,California,CA,Amador,6005,0.387,0.303,0.612,0.593,0.396,0.006,0.156,-120.656960,38.445831,3288
189,California,CA,Butte,6007,0.607,0.327,0.776,0.758,0.057,0.111,0.235,-121.600525,39.667278,10386
190,California,CA,Calaveras,6009,0.362,0.639,0.548,0.126,0.230,0.032,0.098,-120.552913,38.205371,1808


In [300]:
# Now we need to add total number of covid19 deaths per county to the ca_df_raw
# add covid death information for each county. 
df_covid_deaths_US = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")

df_covid_deaths_CA = df_covid_deaths_US.loc[df_covid_deaths_US['Province_State'] == "California"]

df_covid_deaths_CA.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,...,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
195,84006001,US,USA,840,6001.0,Alameda,California,US,37.646294,-121.892927,"Alameda, California, US",1671329,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,625,625,625,657,659,659,659,659,681,697,713,729,732,733,736,763,776,776,768,757,778,764,765,788,842,856,886,894,896,906,929,937,959,974,980,982,991,999,1021,1060
196,84006003,US,USA,840,6003.0,Alpine,California,US,38.596786,-119.822359,"Alpine, California, US",1129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
197,84006005,US,USA,840,6005.0,Amador,California,US,38.445831,-120.656960,"Amador, California, US",39752,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,23,24,24,24,24,24,24,24,26,26,26,26,26,26,28,28,29,29,29,29,29,29,31,31,32,32,32,32,33,33,33,33,33,33,33,34,34,34,34,35
198,84006007,US,USA,840,6007.0,Butte,California,US,39.667278,-121.600525,"Butte, California, US",219186,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,88,90,96,96,96,96,96,100,100,102,102,102,111,111,112,116,120,120,121,121,121,121,123,128,130,134,134,134,136,136,136,136,138,138,138,142,142,145,145,147
199,84006009,US,USA,840,6009.0,Calaveras,California,US,38.205371,-120.552913,"Calaveras, California, US",45905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23


In [301]:
# we clean the data and just grab the county name in Combined_Key column
df_covid_deaths_CA['Combined_Key'] = df_covid_deaths_CA['Combined_Key'].apply(lambda x: x.split(",")[0])
df_covid_deaths_CA.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,...,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
195,84006001,US,USA,840,6001.0,Alameda,California,US,37.646294,-121.892927,Alameda,1671329,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,625,625,625,657,659,659,659,659,681,697,713,729,732,733,736,763,776,776,768,757,778,764,765,788,842,856,886,894,896,906,929,937,959,974,980,982,991,999,1021,1060
196,84006003,US,USA,840,6003.0,Alpine,California,US,38.596786,-119.822359,Alpine,1129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
197,84006005,US,USA,840,6005.0,Amador,California,US,38.445831,-120.656960,Amador,39752,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,23,24,24,24,24,24,24,24,26,26,26,26,26,26,28,28,29,29,29,29,29,29,31,31,32,32,32,32,33,33,33,33,33,33,33,34,34,34,34,35
198,84006007,US,USA,840,6007.0,Butte,California,US,39.667278,-121.600525,Butte,219186,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,88,90,96,96,96,96,96,100,100,102,102,102,111,111,112,116,120,120,121,121,121,121,123,128,130,134,134,134,136,136,136,136,138,138,138,142,142,145,145,147
199,84006009,US,USA,840,6009.0,Calaveras,California,US,38.205371,-120.552913,Calaveras,45905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23


In [302]:
# extract rows 'Out of CA', 'Unassigned'
df_covid_deaths_CA.drop(df_covid_deaths_CA.loc[df_covid_deaths_CA['Combined_Key']=="Out of CA"].index, inplace=True)
df_covid_deaths_CA.drop(df_covid_deaths_CA.loc[df_covid_deaths_CA['Combined_Key']=="Unassigned"].index, inplace=True)
len(df_covid_deaths_CA)

58

In [303]:
# creata total_death row
df_covid_deaths_CA['total_deaths']= df_covid_deaths_CA[df_covid_deaths_CA.columns[-1]]
df_covid_deaths_CA.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,...,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,total_deaths
195,84006001,US,USA,840,6001.0,Alameda,California,US,37.646294,-121.892927,Alameda,1671329,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,625,625,657,659,659,659,659,681,697,713,729,732,733,736,763,776,776,768,757,778,764,765,788,842,856,886,894,896,906,929,937,959,974,980,982,991,999,1021,1060,1060
196,84006003,US,USA,840,6003.0,Alpine,California,US,38.596786,-119.822359,Alpine,1129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
197,84006005,US,USA,840,6005.0,Amador,California,US,38.445831,-120.656960,Amador,39752,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,24,24,24,24,24,24,24,26,26,26,26,26,26,28,28,29,29,29,29,29,29,31,31,32,32,32,32,33,33,33,33,33,33,33,34,34,34,34,35,35
198,84006007,US,USA,840,6007.0,Butte,California,US,39.667278,-121.600525,Butte,219186,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,90,96,96,96,96,96,100,100,102,102,102,111,111,112,116,120,120,121,121,121,121,123,128,130,134,134,134,136,136,136,136,138,138,138,142,142,145,145,147,147
199,84006009,US,USA,840,6009.0,Calaveras,California,US,38.205371,-120.552913,Calaveras,45905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23


In [304]:
# Check total deaths CA
df_covid_deaths_CA["total_deaths"].sum()

43690

In [305]:
# Now merge total deaths with df_raw
ca_df_raw["Total Covid19 Deaths"] = df_covid_deaths_CA.total_deaths.values

ca_df_raw.head()

,State,State Abbreviation,County,FIPS (5-digit),THEME 1: Socioeconomic Status,THEME 2:\nHousehold Composition & Disability,THEME 3: \nMinority Status & Language,THEME 4: \nHousing Type & Transportation,THEME 5: Epidemiological Factors,THEME 6: Healthcare System Factors,CCVI SCORE\nHigher = More Vulnerable,Longitude,Latitude,Total Covid19 Cases,Total Covid19 Deaths
186,California,CA,Alameda,6001,0.184,0.028,0.971,0.604,0.093,0.022,0.097,-121.892927,37.646294,78224,1060
187,California,CA,Alpine,6003,0.813,0.787,0.747,0.989,0.288,0.003,0.515,-119.822359,38.596786,72,0
188,California,CA,Amador,6005,0.387,0.303,0.612,0.593,0.396,0.006,0.156,-120.656960,38.445831,3288,35
189,California,CA,Butte,6007,0.607,0.327,0.776,0.758,0.057,0.111,0.235,-121.600525,39.667278,10386,147
190,California,CA,Calaveras,6009,0.362,0.639,0.548,0.126,0.230,0.032,0.098,-120.552913,38.205371,1808,23


In [306]:
# import hospital data 
hospital_raw = pd.read_csv("https://data.ca.gov/dataset/529ac907-6ba1-4cb7-9aae-8966fc96aeef/resource/42d33765-20fd-44b8-a978-b083b7542225/download/hospitals_by_county.csv")
hospital_raw

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
0,Fresno,2020-03-29,8.0,116.0,NaN,NaN,3.0,14.0,22.0
1,Marin,2020-03-29,7.0,13.0,NaN,NaN,2.0,6.0,11.0
2,Tehama,2020-03-29,0.0,0.0,NaN,NaN,0.0,0.0,2.0
3,Contra Costa,2020-03-29,23.0,62.0,NaN,NaN,8.0,15.0,62.0
4,San Mateo,2020-03-29,46.0,27.0,NaN,NaN,16.0,10.0,28.0
...,...,...,...,...,...,...,...,...,...
17576,Modoc,2021-02-05,2.0,0.0,2.0,12.0,0.0,0.0,0.0
17577,Mono,2021-02-05,0.0,0.0,0.0,17.0,0.0,0.0,2.0
17578,Imperial,2021-02-05,49.0,0.0,49.0,234.0,22.0,0.0,4.0
17579,Monterey,2021-02-05,109.0,1.0,110.0,720.0,25.0,0.0,18.0


In [307]:
# First we need to change todays_date column to a datetime column
hospital_raw['todays_date'] = pd.to_datetime(hospital_raw['todays_date'])
most_recent_date = hospital_raw["todays_date"].max()
most_recent_date.date()

# hospital_raw = hospital_raw.loc[hospital_raw['todays_date'] == most_recent_date ]
# hospital_raw
x = hospital_raw.groupby(by=["county"]).sum()
len(x.index)

lst3 = list(x.index)

new_list1 = [
    list(set(lst2).difference(lst3))
]
new_list1

# No hospital data from Alpine and Sierra

[['Sierra', 'Alpine']]

In [308]:
# we grab the data from the most recent date
hospital_raw = hospital_raw.loc[hospital_raw['todays_date'] == most_recent_date ]

# create a columns list 
columns_lst = list(hospital_raw.columns)

# create an empty list to add the cities that don't have hospital data.
empty_list = []

# loop over new_list1[0]
for i in new_list1[0]:
  data = [str(i),most_recent_date,None,None,None,None,None,None,None]
  empty_list.append(data)

# create a second dataframe
hospital_raw_df2 = pd.DataFrame(empty_list,columns=columns_lst)

# append the second dataframe to hospital_raw
hospital_raw = hospital_raw.append(hospital_raw_df2, ignore_index=True)

# we sort the hospital data by county name
hospital_raw = hospital_raw.sort_values("county")

# length should be 58
len(hospital_raw)

58

In [309]:
# check hospital data now
hospital_raw.head()

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
32,Alameda,2021-02-05,294,34,328,2498,85,2,73
57,Alpine,2021-02-05,None,None,None,None,None,None,None
26,Amador,2021-02-05,7,1,8,52,2,0,2
20,Butte,2021-02-05,31,0,31,451,8,0,9
48,Calaveras,2021-02-05,5,0,5,33,1,0,5


In [310]:
# now we add the hospital data to our original ca_df_raw

# Now merge the hospital data to each county in ca_df_raw
ca_df_raw["todays_date"] = hospital_raw.todays_date.values
ca_df_raw["hospitalized_covid_confirmed_patients"] = hospital_raw.hospitalized_covid_confirmed_patients.values
ca_df_raw["hospitalized_suspected_covid_patients"] = hospital_raw.hospitalized_suspected_covid_patients.values
ca_df_raw["hospitalized_covid_patients"] = hospital_raw.hospitalized_covid_patients.values
ca_df_raw["all_hospital_beds"] = hospital_raw.all_hospital_beds.values
ca_df_raw["icu_covid_confirmed_patients"] = hospital_raw.icu_covid_confirmed_patients.values
ca_df_raw["icu_suspected_covid_patients"] = hospital_raw.icu_suspected_covid_patients.values
ca_df_raw["icu_available_beds"] = hospital_raw.icu_available_beds.values

ca_df_raw.head()

,State,State Abbreviation,County,FIPS (5-digit),THEME 1: Socioeconomic Status,THEME 2:\nHousehold Composition & Disability,THEME 3: \nMinority Status & Language,THEME 4: \nHousing Type & Transportation,THEME 5: Epidemiological Factors,THEME 6: Healthcare System Factors,CCVI SCORE\nHigher = More Vulnerable,Longitude,Latitude,Total Covid19 Cases,Total Covid19 Deaths,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
186,California,CA,Alameda,6001,0.184,0.028,0.971,0.604,0.093,0.022,0.097,-121.892927,37.646294,78224,1060,2021-02-05,294,34,328,2498,85,2,73
187,California,CA,Alpine,6003,0.813,0.787,0.747,0.989,0.288,0.003,0.515,-119.822359,38.596786,72,0,2021-02-05,None,None,None,None,None,None,None
188,California,CA,Amador,6005,0.387,0.303,0.612,0.593,0.396,0.006,0.156,-120.656960,38.445831,3288,35,2021-02-05,7,1,8,52,2,0,2
189,California,CA,Butte,6007,0.607,0.327,0.776,0.758,0.057,0.111,0.235,-121.600525,39.667278,10386,147,2021-02-05,31,0,31,451,8,0,9
190,California,CA,Calaveras,6009,0.362,0.639,0.548,0.126,0.230,0.032,0.098,-120.552913,38.205371,1808,23,2021-02-05,5,0,5,33,1,0,5


In [311]:
# cleaned data as of now 
ca_df_raw.head()

,State,State Abbreviation,County,FIPS (5-digit),THEME 1: Socioeconomic Status,THEME 2:\nHousehold Composition & Disability,THEME 3: \nMinority Status & Language,THEME 4: \nHousing Type & Transportation,THEME 5: Epidemiological Factors,THEME 6: Healthcare System Factors,CCVI SCORE\nHigher = More Vulnerable,Longitude,Latitude,Total Covid19 Cases,Total Covid19 Deaths,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
186,California,CA,Alameda,6001,0.184,0.028,0.971,0.604,0.093,0.022,0.097,-121.892927,37.646294,78224,1060,2021-02-05,294,34,328,2498,85,2,73
187,California,CA,Alpine,6003,0.813,0.787,0.747,0.989,0.288,0.003,0.515,-119.822359,38.596786,72,0,2021-02-05,None,None,None,None,None,None,None
188,California,CA,Amador,6005,0.387,0.303,0.612,0.593,0.396,0.006,0.156,-120.656960,38.445831,3288,35,2021-02-05,7,1,8,52,2,0,2
189,California,CA,Butte,6007,0.607,0.327,0.776,0.758,0.057,0.111,0.235,-121.600525,39.667278,10386,147,2021-02-05,31,0,31,451,8,0,9
190,California,CA,Calaveras,6009,0.362,0.639,0.548,0.126,0.230,0.032,0.098,-120.552913,38.205371,1808,23,2021-02-05,5,0,5,33,1,0,5


In [312]:
# We grab the population data from the year 2020.
# cities_pop = pd.read_html('https://www.california-demographics.com/counties_by_population', header = 0)
cities_pop = pd.read_html('https://worldpopulationreview.com/us-counties/states/ca', header = 0)
df_pop = cities_pop[0]
df_pop = df_pop.rename(columns={"2021 Population": "Population"})

city_lst = []
for i in df_pop.Name.values:
  spl_string = i.split()
  rm = spl_string[:-1]
  listToStr = ' '.join([str(elem) for elem in rm])
  city_lst.append(listToStr)

df_pop['county'] = city_lst
df_pop = df_pop.drop(columns=['Name'],axis=1)
df_pop

# # We create a dictionary where keys are city and the value is the population number.
pop_dict = pd.Series(df_pop.Population.values,index=df_pop.county).to_dict()
# pop_dict.pop('United States Census Bureau. Annual Estimates of the Resident Population: April 1, 2010 to July 1, 2019. U.S. Census Bureau, Population Division. Web. May 2020. http://www.census.gov/. United States Census Bureau. B01001 SEX BY AGE, 2019 American Community Survey 5-Year Estimates. U.S. Census Bureau, American Community Survey Office. Web. 10 December 2020.')
pop_dict

{'Alameda': 1680480,
 'Alpine': 1209,
 'Amador': 40446,
 'Butte': 196880,
 'Calaveras': 46319,
 'Colusa': 21805,
 'Contra Costa': 1159540,
 'Del Norte': 27956,
 'El Dorado': 197037,
 'Fresno': 1013400,
 'Glenn': 29245,
 'Humboldt': 134186,
 'Imperial': 180599,
 'Inyo': 18225,
 'Kern': 913090,
 'Kings': 156056,
 'Lake': 64524,
 'Lassen': 30483,
 'Los Angeles': 9969510,
 'Madera': 158217,
 'Marin': 257154,
 'Mariposa': 16799,
 'Mendocino': 85445,
 'Merced': 284738,
 'Modoc': 8923,
 'Mono': 14526,
 'Monterey': 434283,
 'Napa': 135654,
 'Nevada': 100249,
 'Orange': 3175130,
 'Placer': 410327,
 'Plumas': 18939,
 'Riverside': 2520060,
 'Sacramento': 1578680,
 'San Benito': 65490,
 'San Bernardino': 2206750,
 'San Diego': 3347270,
 'San Francisco': 883255,
 'San Joaquin': 781462,
 'San Luis Obispo': 282625,
 'San Mateo': 762357,
 'Santa Barbara': 447937,
 'Santa Clara': 1918880,
 'Santa Cruz': 271957,
 'Shasta': 180822,
 'Sierra': 3021,
 'Siskiyou': 43517,
 'Solano': 451479,
 'Sonoma': 485722

In [313]:
# We create a list of the locations in the dataset
df_loc_lst = list(ca_df_raw.County.values)

pop_lst = []

# Iterated of the list of locations in the dataset.
for i in df_loc_lst:
    try:
        # Append the population to the pop_lst
        pop_lst.append(pop_dict[i])
    except KeyError:
        # If it doesn't have the city in the data that we scraped we just add 0
        pop_lst.append(0)
        continue

# Create a new column
ca_df_raw["population"] = pop_lst
ca_df_raw.head()

,State,State Abbreviation,County,FIPS (5-digit),THEME 1: Socioeconomic Status,THEME 2:\nHousehold Composition & Disability,THEME 3: \nMinority Status & Language,THEME 4: \nHousing Type & Transportation,THEME 5: Epidemiological Factors,THEME 6: Healthcare System Factors,CCVI SCORE\nHigher = More Vulnerable,Longitude,Latitude,Total Covid19 Cases,Total Covid19 Deaths,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds,population
186,California,CA,Alameda,6001,0.184,0.028,0.971,0.604,0.093,0.022,0.097,-121.892927,37.646294,78224,1060,2021-02-05,294,34,328,2498,85,2,73,1680480
187,California,CA,Alpine,6003,0.813,0.787,0.747,0.989,0.288,0.003,0.515,-119.822359,38.596786,72,0,2021-02-05,None,None,None,None,None,None,None,1209
188,California,CA,Amador,6005,0.387,0.303,0.612,0.593,0.396,0.006,0.156,-120.656960,38.445831,3288,35,2021-02-05,7,1,8,52,2,0,2,40446
189,California,CA,Butte,6007,0.607,0.327,0.776,0.758,0.057,0.111,0.235,-121.600525,39.667278,10386,147,2021-02-05,31,0,31,451,8,0,9,196880
190,California,CA,Calaveras,6009,0.362,0.639,0.548,0.126,0.230,0.032,0.098,-120.552913,38.205371,1808,23,2021-02-05,5,0,5,33,1,0,5,46319


In [314]:
# check df data types
ca_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 186 to 243
Data columns (total 24 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   State                                        58 non-null     object        
 1   State Abbreviation                           58 non-null     object        
 2   County                                       58 non-null     object        
 3   FIPS (5-digit)                               58 non-null     int64         
 4   THEME 1: Socioeconomic Status                58 non-null     float64       
 5   THEME 2:
Household Composition & Disability  58 non-null     float64       
 6   THEME 3: 
Minority Status & Language         58 non-null     float64       
 7   THEME 4: 
Housing Type & Transportation      58 non-null     float64       
 8   THEME 5: Epidemiological Factors             58 non-null     float64       
 9 

In [315]:
# run basic stats
ca_df_raw.describe()

,FIPS (5-digit),THEME 1: Socioeconomic Status,THEME 2:\nHousehold Composition & Disability,THEME 3: \nMinority Status & Language,THEME 4: \nHousing Type & Transportation,THEME 5: Epidemiological Factors,THEME 6: Healthcare System Factors,CCVI SCORE\nHigher = More Vulnerable,Longitude,Latitude,Total Covid19 Cases,Total Covid19 Deaths,population
count,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,5.800000e+01,58.000000,5.800000e+01
mean,6058.000000,0.508414,0.346638,0.845431,0.608897,0.160224,0.033810,0.252017,-120.728594,37.843962,5.861629e+04,753.275862,6.829913e+05
std,33.773757,0.269966,0.271088,0.167797,0.232298,0.188195,0.034613,0.191745,1.922509,2.148394,1.592544e+05,2398.353747,1.462337e+06
min,6001.000000,0.016000,0.007000,0.336000,0.121000,0.006000,0.000000,0.008000,-123.897406,33.034846,7.200000e+01,0.000000,1.209000e+03
25%,6029.500000,0.254000,0.109750,0.769250,0.463250,0.057000,0.006000,0.098750,-122.031429,36.641646,3.002250e+03,31.250000,4.840425e+04
50%,6058.000000,0.559000,0.312500,0.940500,0.639000,0.103500,0.021000,0.203000,-120.918298,38.048832,1.227050e+04,156.500000,1.888510e+05
75%,6086.500000,0.748500,0.593000,0.969500,0.778000,0.163000,0.060500,0.361250,-119.843603,39.241624,4.449125e+04,558.000000,7.106998e+05
max,6115.000000,0.974000,0.892000,0.997000,0.989000,0.843000,0.161000,0.812000,-115.366904,41.742283,1.138970e+06,17772.000000,9.969510e+06


In [316]:
ca_df_raw.columns

Index(['State', 'State Abbreviation', 'County', 'FIPS (5-digit)',
       'THEME 1: Socioeconomic Status',
       'THEME 2:\nHousehold Composition & Disability',
       'THEME 3: \nMinority Status & Language',
       'THEME 4: \nHousing Type & Transportation',
       'THEME 5: Epidemiological Factors',
       'THEME 6: Healthcare System Factors',
       'CCVI SCORE\nHigher = More Vulnerable', 'Longitude', 'Latitude',
       'Total Covid19 Cases', 'Total Covid19 Deaths', 'todays_date',
       'hospitalized_covid_confirmed_patients',
       'hospitalized_suspected_covid_patients', 'hospitalized_covid_patients',
       'all_hospital_beds', 'icu_covid_confirmed_patients',
       'icu_suspected_covid_patients', 'icu_available_beds', 'population'],
      dtype='object')

In [317]:
# rename cols for easier referencing
ca_df_raw.rename(columns={
    'State': 'state',
    'State Abbreviation': 'abbreviation',
    'County': 'county',
    'FIPS (5-digit)': 'fips',
    'THEME 1: Socioeconomic Status':'t1_socioecon',
    'THEME 2:\nHousehold Composition & Disability': 't2_household',
    'THEME 3: \nMinority Status & Language' : 't3_minority',
    'THEME 4: \nHousing Type & Transportation' : 't4_housing',
       'THEME 5: Epidemiological Factors' : 'epidim',
       'THEME 6: Healthcare System Factors' : 'healthcare_sys',
       'CCVI SCORE\nHigher = More Vulnerable' : 'ccvi_score',
       'Longitude' : 'long', 
       'Latitude' : 'lat',
       'Total Covid19 Cases': 'covid_total', 
       'Total Covid19 Deaths' : 'covid_deaths',
       'hospitalized_covid_confirmed_patients' : 'covid_confirmed',
       'hospitalized_suspected_covid_patients' : 'covid_suspected',
       'hospitalized_covid_patients' : 'covid_total_hospitalized'}, 
       inplace = True, errors='raise')

In [318]:
# add columns for per 100k 
ca_df_raw['covid_cases_per_100k'] = ca_df_raw.apply(lambda x: (x.covid_total/x.population) * 100000, axis = 1)
ca_df_raw['covid_deaths_per_100k'] = ca_df_raw.apply(lambda x: (x.covid_deaths/x.population) * 100000, axis = 1)

In [319]:
# check # missing values
ca_df_raw.isnull().sum()

state                           0
abbreviation                    0
county                          0
fips                            0
t1_socioecon                    0
t2_household                    0
t3_minority                     0
t4_housing                      0
epidim                          0
healthcare_sys                  0
ccvi_score                      0
long                            0
lat                             0
covid_total                     0
covid_deaths                    0
todays_date                     0
covid_confirmed                 2
covid_suspected                 2
covid_total_hospitalized        2
all_hospital_beds               2
icu_covid_confirmed_patients    2
icu_suspected_covid_patients    2
icu_available_beds              2
population                      0
covid_cases_per_100k            0
covid_deaths_per_100k           0
dtype: int64

In [320]:
ca_df_raw.loc[(ca_df_raw.covid_total < 100)]

,state,abbreviation,county,fips,t1_socioecon,t2_household,t3_minority,t4_housing,epidim,healthcare_sys,ccvi_score,long,lat,covid_total,covid_deaths,todays_date,covid_confirmed,covid_suspected,covid_total_hospitalized,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds,population,covid_cases_per_100k,covid_deaths_per_100k
187,California,CA,Alpine,6003,0.813,0.787,0.747,0.989,0.288,0.003,0.515,-119.822359,38.596786,72,0,2021-02-05,None,None,None,None,None,None,None,1209,5955.334988,0.0
231,California,CA,Sierra,6091,0.149,0.101,0.336,0.587,0.244,0.005,0.017,-120.520070,39.577135,95,0,2021-02-05,None,None,None,None,None,None,None,3021,3144.654088,0.0


In [321]:
# convert "None" values to numeric
ca_df_raw.fillna(0)

,state,abbreviation,county,fips,t1_socioecon,t2_household,t3_minority,t4_housing,epidim,healthcare_sys,ccvi_score,long,lat,covid_total,covid_deaths,todays_date,covid_confirmed,covid_suspected,covid_total_hospitalized,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds,population,covid_cases_per_100k,covid_deaths_per_100k
186,California,CA,Alameda,6001,0.184,0.028,0.971,0.604,0.093,0.022,0.097,-121.892927,37.646294,78224,1060,2021-02-05,294.0,34.0,328.0,2498.0,85.0,2.0,73.0,1680480,4654.860516,63.077216
187,California,CA,Alpine,6003,0.813,0.787,0.747,0.989,0.288,0.003,0.515,-119.822359,38.596786,72,0,2021-02-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1209,5955.334988,0.000000
188,California,CA,Amador,6005,0.387,0.303,0.612,0.593,0.396,0.006,0.156,-120.656960,38.445831,3288,35,2021-02-05,7.0,1.0,8.0,52.0,2.0,0.0,2.0,40446,8129.357662,86.535133
189,California,CA,Butte,6007,0.607,0.327,0.776,0.758,0.057,0.111,0.235,-121.600525,39.667278,10386,147,2021-02-05,31.0,0.0,31.0,451.0,8.0,0.0,9.0,196880,5275.294596,74.664770
190,California,CA,Calaveras,6009,0.362,0.639,0.548,0.126,0.230,0.032,0.098,-120.552913,38.205371,1808,23,2021-02-05,5.0,0.0,5.0,33.0,1.0,0.0,5.0,46319,3903.365789,49.655649
191,California,CA,Colusa,6011,0.690,0.577,0.989,0.888,0.067,0.007,0.420,-122.233173,39.178820,2004,11,2021-02-05,4.0,0.0,4.0,48.0,0.0,0.0,5.0,21805,9190.552626,50.447145
192,California,CA,Contra Costa,6013,0.197,0.112,0.941,0.259,0.114,0.014,0.095,-121.928953,37.919235,58462,552,2021-02-05,166.0,21.0,187.0,1340.0,45.0,3.0,35.0,1159540,5041.826931,47.605085
193,California,CA,Del Norte,6015,0.791,0.598,0.834,0.921,0.070,0.046,0.359,-123.897406,41.742283,894,3,2021-02-05,0.0,0.0,0.0,52.0,0.0,0.0,3.0,27956,3197.882387,10.731149
194,California,CA,El Dorado,6017,0.145,0.142,0.663,0.149,0.103,0.005,0.021,-120.523317,38.779660,8647,86,2021-02-05,6.0,0.0,6.0,190.0,1.0,0.0,8.0,197037,4388.515863,43.646625
195,California,CA,Fresno,6019,0.810,0.556,0.981,0.668,0.055,0.085,0.457,-119.646695,36.757339,90126,1224,2021-02-05,405.0,29.0,434.0,1525.0,97.0,1.0,7.0,1013400,8893.428064,120.781528


In [322]:
ca_df_raw.icu_available_beds.describe()

count     56.0
unique    31.0
top        0.0
freq      11.0
Name: icu_available_beds, dtype: float64

In [323]:
covid_total_series = pd.Series(ca_df_raw.covid_total.values,ca_df_raw.county.values)
covid_deaths_series = pd.Series(ca_df_raw.covid_deaths.values,ca_df_raw.county.values)

### HPI Index

#### Load & Clean Data

In [324]:
# import HPI index data
# you can download it from here: https://healthyplacesindex.org/data-reports
df_hpi_raw = pd.read_csv('https://storage.googleapis.com/hpi_index/HPI2_MasterFile_2019-04-24.csv', encoding= 'unicode_escape')
df_hpi_raw

,CensusTract,pop2010,pct2010gq,City,ZIP,County_FIPS,County_Name,UrbanType,hpi2score,hpi2_pctile_pos,hpi2_pctile_neg,hpi_top25pct,quintiles,quartiles,economic,economic_pctile,education,education_pctile,housing,houseing_pctile,healthcareaccess,healthcareaccess_pctile,neighborhood,neighborhood_pctile,pollution,pollution_pctile,transportation,transportation_pctile,social,social_pctile,insured,insured_pctile,uncrowded,uncrowded_pctile,homeownership,homeownership_pctile,automobile,automobile_pctile,commute,commute_pctile,...,income_pctile,retail,retail_pctile,parkaccess,parkaccess_pctile,treecanopy,treecanopy_pctile,alcoffsale,alcoffsale_pctile,voting,voting_pctile,ownsevere,ownsevere_pctile,rentsevere,rentsevere_pctile,houserepair,houserepair_pctile,twoparents,twoparents_pctile,supermkts,supermkts_pctile,ozone,ozone_pctile,pm25,pm25_pctile,dieselpm,dieselpm_pctile,h20contam,h20contam_pctile,LEB,LEB_pctile,white_pct,black_pct,asian_pct,latino_pct,multiple_pct,NativeAm_pct,PacificIsl_pct,other_pct,version
0,6001400100,2937,0.1,Oakland,94704.0,6001,Alameda,urban_area,1.182028,99.114590,0.885410,Yes,1.0,1.0,1.735963,98.691133,0.948968,90.093674,0.730552,90.003850,1.285242,93.814962,0.832170,98.395996,0.878125,92.377775,0.616855,95.226485,1.336602,96.881817,95.0,93.814962,99.3,89.464904,85.5,92.005646,98.6,87.475940,15.284553,82.882074,...,99.101758,9.587563,97.600411,100.00000,81.355062,36.433798,97.420762,73.169901,64.121648,82.781682,93.224689,5.092593,90.068010,27.272727,45.784679,100.000000,80.816117,93.074792,92.480431,1.319206,13.396638,0.029592,96.086231,8.697944,74.759399,27.436933,18.927242,70.599583,97.202618,86.985506,96.663673,70.752469,4.766769,15.526047,3.983657,3.779367,0.068097,0.272387,0.851209,Wed13Dec17
1,6001400200,1974,3.5,Oakland,94618.0,6001,Alameda,urban_area,1.325874,99.704863,0.295137,Yes,1.0,1.0,1.808761,99.037598,1.137725,94.649044,0.959728,96.830489,1.004971,83.985628,0.503232,94.379571,0.657261,85.628128,1.494366,99.653535,1.227116,94.687540,91.6,83.985628,100.0,96.933145,63.6,59.502117,94.1,46.028487,41.950758,96.791993,...,96.650841,2.448043,82.651097,91.79331,50.263057,19.708371,92.467599,28.419453,26.831772,85.492572,97.805723,3.883495,94.187091,2.666667,97.895547,99.386503,51.148467,87.012987,77.993071,100.000000,94.251251,0.029592,96.086231,8.697944,74.759399,42.190000,5.569100,70.599583,97.202618,86.957616,96.612344,78.318136,1.570415,7.396150,7.649443,4.812563,0.101317,0.000000,0.151976,Wed13Dec17
2,6001400300,4865,0.8,Oakland,94618.0,6001,Alameda,urban_area,0.925666,95.932247,4.067753,Yes,1.0,1.0,1.014492,87.244963,1.348314,98.165020,0.603628,84.088284,1.268756,93.186193,-0.291977,19.940973,0.657261,85.628128,0.822268,97.202618,1.078500,90.966252,94.8,93.186193,100.0,96.933145,44.3,33.850892,85.5,14.115232,37.873988,96.047735,...,74.990376,1.890912,76.478891,33.25797,15.834723,5.838530,57.320672,16.690647,19.312203,82.086597,91.813166,7.359307,79.070961,12.177122,88.771975,100.000000,80.816117,87.221571,78.429360,92.745177,80.354164,0.029592,96.086231,8.697944,74.759399,42.190000,5.569100,70.599583,97.202618,86.462942,95.136661,66.927030,10.524152,8.612539,8.201439,5.056526,0.164440,0.102775,0.411100,Wed13Dec17
3,6001400400,3703,0.9,Oakland,94609.0,6001,Alameda,urban_area,1.117603,98.537149,1.462851,Yes,1.0,1.0,1.277657,93.622482,0.939501,89.914025,0.356477,68.381881,1.301729,94.430900,0.070027,57.936610,0.657261,85.628128,1.756657,99.910176,1.295417,96.253048,95.2,94.430900,99.2,88.258694,37.9,26.305659,92.9,38.611574,49.724518,98.011036,...,86.500706,1.828020,75.619145,84.22900,42.730656,11.378516,82.997562,0.000000,4.516874,83.732539,95.175157,13.103448,42.640832,11.926605,89.246760,99.449036,53.432568,90.876565,88.374182,97.844466,84.588733,0.029592,96.086231,8.697944,74.759399,42.190000,5.569100,70.599583,97.202618,82.710897,68.959322,65.460437,12.098299,7.291385,8.965703,5.320011,0.135026,0.162031,0.567108,Wed13Dec17
4,6001400500,3517,3.6,Oakland,94609.0,6001,Alameda,urban_area,0.562799,83.3

In [325]:
# view column data types
df_hpi_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8057 entries, 0 to 8056
Data columns (total 91 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CensusTract              8057 non-null   int64  
 1   pop2010                  8057 non-null   int64  
 2   pct2010gq                8024 non-null   float64
 3   City                     6972 non-null   object 
 4   ZIP                      8035 non-null   float64
 5   County_FIPS              8057 non-null   int64  
 6   County_Name              8057 non-null   object 
 7   UrbanType                8057 non-null   object 
 8   hpi2score                7793 non-null   float64
 9   hpi2_pctile_pos          7793 non-null   float64
 10  hpi2_pctile_neg          7793 non-null   float64
 11  hpi_top25pct             7793 non-null   object 
 12  quintiles                7793 non-null   float64
 13  quartiles                7793 non-null   float64
 14  economic                

In [326]:
# run basic stats
df_hpi_raw.describe()

,CensusTract,pop2010,pct2010gq,ZIP,County_FIPS,hpi2score,hpi2_pctile_pos,hpi2_pctile_neg,quintiles,quartiles,economic,economic_pctile,education,education_pctile,housing,houseing_pctile,healthcareaccess,healthcareaccess_pctile,neighborhood,neighborhood_pctile,pollution,pollution_pctile,transportation,transportation_pctile,social,social_pctile,insured,insured_pctile,uncrowded,uncrowded_pctile,homeownership,homeownership_pctile,automobile,automobile_pctile,commute,commute_pctile,inpreschool,inpreschool_pctile,inhighschool,inhighschool_pctile,...,income,income_pctile,retail,retail_pctile,parkaccess,parkaccess_pctile,treecanopy,treecanopy_pctile,alcoffsale,alcoffsale_pctile,voting,voting_pctile,ownsevere,ownsevere_pctile,rentsevere,rentsevere_pctile,houserepair,houserepair_pctile,twoparents,twoparents_pctile,supermkts,supermkts_pctile,ozone,ozone_pctile,pm25,pm25_pctile,dieselpm,dieselpm_pctile,h20contam,h20contam_pctile,LEB,LEB_pctile,white_pct,black_pct,asian_pct,latino_pct,multiple_pct,NativeAm_pct,PacificIsl_pct,other_pct
count,8.057000e+03,8057.000000,8024.000000,8035.000000,8057.000000,7.793000e+03,7793.000000,7793.000000,7793.000000,7793.000000,7.793000e+03,7793.000000,7.793000e+03,7793.000000,7.793000e+03,7793.000000,7.793000e+03,7793.000000,7.793000e+03,7793.000000,7.793000e+03,7793.000000,7.793000e+03,7793.000000,7.793000e+03,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,...,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,7793.000000,8057.000000,8057.000000,8057.000000,8057.000000,8057.000000,8057.000000,8057.000000,8057.000000
mean,6.054898e+09,4623.799926,2.307515,92836.608214,6054.690952,2.438298e-12,50.006416,49.993584,3.000000,2.500064,5.646001e-12,50.006416,1.411520e-12,50.006111,-1.283313e-12,50.006416,1.283205e-13,50.003169,-1.411457e-12,50.006416,1.013758e-11,50.005838,2.309760e-12,50.006349,1.154779e-12,50.006416,79.408623,50.003169,90.698576,50.003103,54.337842,50.003304,92.098127,50.002863,9.654941,50.006042,52.267040,50.003388,97.785780,76.226906,...,67385.515155,50.005502,1.949688,50.006416,74.356703,50.004012,8.126024,50.006414,54.182934,50.005445,67.711987,50.006416,13.323066,50.004000,26.288570,50.003986,98.647874,50.002347,76.420797,50.006085,49.747290,50.006416,0.047431,49.996880,10.392395,49.997292,19.225058,49.994518,472.535176,49.994858,80.905281,50.006416,41.399286,5.858864,12.423550,36.288136,2.599598,0.460792,0.332727,0.227466
std,2.653142e+07,1984.696591,9.574740,3445.164575,26.609992,5.487532e-01,28.869366,28.869366,1.414123,1.118048,8.704671e-01,28.869366,6.825760e-01,28.869144,6.491728e-01,28.869366,1.000000e+00,28.869258,3.871541e-01,28.869366,6.334197e-01,28.869435,3.696637e-01,28.869398,8.297582e-01,28.869366,12.131079,28.869258,9.805062,28.865514,23.564124,28.869302,8.632851,28.868033,11.714788,28.869568,26.789801,28.860150,5.401372,38.755211,...,32776.843403,28.869438,7.922636,28.869366,32.719631,28.107014,8.888843,28.869366,33.062726,28.855918,10.869562,28.869366,8.615520,28.869376,11.563002,28.869253,2.319737,28.040426,12.942265,28.869308,35.983316,28.845761,0.010272,28.794412,2.580386,28.766065,16.699284,28.868945,249.492410,28.867119,3.465141,28.869366,27.003564,9.384340,14.655063,26.290103,1.480804,1.416903,0.584286,0.235862
min,6.001400e+09,0.000000,0.000000,32.000000,6001.000000,-1.994633e+00,0.012832,0.000000,1.000000,1.000000,-2.772752e+00,0.012832,-4.727192e+00,0.012832,-6.071828e+00,0.012832,-4.551007e+00,0.012832,-1.161717e+00,0.012832,-3.431559e+00,0.012832,-2.786433e+00,0.012832,-3.421043e+00,0.012832,24.200000,0.012832,29.60

In [327]:
df_hpi_raw.shape

(8057, 91)

In [328]:
# check unique values
df_hpi_raw['hpi2score'].unique()

array([ 1.1820283 ,  1.3258738 ,  0.9256657 , ..., -0.79174168,
        0.16293131, -0.21838948])

#### Check Missing Values

In [329]:
# set display to max length of cols to see all counts
pd.set_option('display.max_rows', 91)

# check # missing values
hpi_missing_cols = df_hpi_raw.isnull().sum()
print(hpi_missing_cols.sort_values(ascending=False))

City                       1085
bachelorsed_pctile          264
commute_pctile              264
insured_pctile              264
uncrowded                   264
uncrowded_pctile            264
homeownership               264
homeownership_pctile        264
automobile                  264
automobile_pctile           264
commute                     264
inpreschool                 264
social_pctile               264
inpreschool_pctile          264
inhighschool                264
inhighschool_pctile         264
bachelorsed                 264
hpi2score                   264
employed                    264
employed_pctile             264
insured                     264
social                      264
abovepoverty_pctile         264
housing                     264
hpi_top25pct                264
quintiles                   264
quartiles                   264
economic                    264
economic_pctile             264
education                   264
education_pctile            264
houseing

In [330]:
# check if any rows are missing more than 70%
seventy_percent = 8057 * .7
df_hpi_raw.columns[df_hpi_raw.isnull().sum() > seventy_percent]

Index([], dtype='object')

In [331]:
# view column names
hpi_raw_columns = df_hpi_raw.columns
print(hpi_raw_columns)

Index(['CensusTract', 'pop2010', 'pct2010gq', 'City', 'ZIP', 'County_FIPS',
       'County_Name', 'UrbanType', 'hpi2score', 'hpi2_pctile_pos',
       'hpi2_pctile_neg', 'hpi_top25pct', 'quintiles', 'quartiles', 'economic',
       'economic_pctile', 'education', 'education_pctile', 'housing',
       'houseing_pctile', 'healthcareaccess', 'healthcareaccess_pctile',
       'neighborhood', 'neighborhood_pctile', 'pollution', 'pollution_pctile',
       'transportation', 'transportation_pctile', 'social', 'social_pctile',
       'insured', 'insured_pctile', 'uncrowded', 'uncrowded_pctile',
       'homeownership', 'homeownership_pctile', 'automobile',
       'automobile_pctile', 'commute', 'commute_pctile', 'inpreschool',
       'inpreschool_pctile', 'inhighschool', 'inhighschool_pctile',
       'bachelorsed', 'bachelorsed_pctile', 'employed', 'employed_pctile',
       'abovepoverty', 'abovepoverty_pctile', 'income', 'income_pctile',
       'retail', 'retail_pctile', 'parkaccess', 'parkaccess

In [332]:
# drop columns we don't need
cols_to_keep = ['City', 'ZIP', 'County_FIPS', 'County_Name', 'hpi2_pctile_pos',
       'hpi2_pctile_neg', 'hpi_top25pct', 'quintiles', 'quartiles', 
       'healthcareaccess', 'healthcareaccess_pctile']
cols_to_drop = ['UrbanType', 'economic',
       'economic_pctile', 'education', 'education_pctile', 'housing',
       'houseing_pctile', 'healthcareaccess', 'healthcareaccess_pctile',
       'neighborhood', 'neighborhood_pctile', 'pollution', 'pollution_pctile',
       'transportation', 'transportation_pctile', 'social', 'social_pctile',
       'insured', 'insured_pctile', 'uncrowded', 'uncrowded_pctile',
       'homeownership', 'homeownership_pctile', 'automobile',
       'automobile_pctile', 'commute', 'commute_pctile', 'inpreschool',
       'inpreschool_pctile', 'inhighschool', 'inhighschool_pctile',
       'bachelorsed', 'bachelorsed_pctile', 'employed', 'employed_pctile',
       'abovepoverty', 'abovepoverty_pctile', 'income', 'income_pctile',
       'retail', 'retail_pctile', 'parkaccess', 'parkaccess_pctile',
       'treecanopy', 'treecanopy_pctile', 'alcoffsale', 'alcoffsale_pctile',
       'voting', 'voting_pctile', 'ownsevere', 'ownsevere_pctile',
       'rentsevere', 'rentsevere_pctile', 'houserepair', 'houserepair_pctile',
       'twoparents', 'twoparents_pctile', 'supermkts', 'supermkts_pctile',
       'ozone', 'ozone_pctile', 'pm25', 'pm25_pctile', 'dieselpm',
       'dieselpm_pctile', 'h20contam', 'h20contam_pctile', 'LEB', 'LEB_pctile',
       'white_pct', 'black_pct', 'asian_pct', 'latino_pct', 'multiple_pct',
       'NativeAm_pct', 'PacificIsl_pct', 'other_pct', 'version']
df_hpi = df_hpi_raw.drop([], axis=1)

No rows are missing more than 70 percent of values. The highest percent of missing values is the city column which is missing 13%. 

In [333]:
# view missing zip code rows
df_hpi[df_hpi['ZIP'].isnull()]

,CensusTract,pop2010,pct2010gq,City,ZIP,County_FIPS,County_Name,UrbanType,hpi2score,hpi2_pctile_pos,hpi2_pctile_neg,hpi_top25pct,quintiles,quartiles,economic,economic_pctile,education,education_pctile,housing,houseing_pctile,healthcareaccess,healthcareaccess_pctile,neighborhood,neighborhood_pctile,pollution,pollution_pctile,transportation,transportation_pctile,social,social_pctile,insured,insured_pctile,uncrowded,uncrowded_pctile,homeownership,homeownership_pctile,automobile,automobile_pctile,commute,commute_pctile,...,income_pctile,retail,retail_pctile,parkaccess,parkaccess_pctile,treecanopy,treecanopy_pctile,alcoffsale,alcoffsale_pctile,voting,voting_pctile,ownsevere,ownsevere_pctile,rentsevere,rentsevere_pctile,houserepair,houserepair_pctile,twoparents,twoparents_pctile,supermkts,supermkts_pctile,ozone,ozone_pctile,pm25,pm25_pctile,dieselpm,dieselpm_pctile,h20contam,h20contam_pctile,LEB,LEB_pctile,white_pct,black_pct,asian_pct,latino_pct,multiple_pct,NativeAm_pct,PacificIsl_pct,other_pct,version
360,6001990000,0,NaN,Fremont,NaN,6001,Alameda,no_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Wed13Dec17
644,6013990000,0,NaN,NaN,NaN,6013,Contra Costa,no_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Wed13Dec17
652,6015990000,0,NaN,NaN,NaN,6015,Del Norte,no_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Wed13Dec17
695,6017990000,0,NaN,NaN,NaN,6017,El Dorado,no_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Wed13Dec17
931,6023990100,0,NaN,NaN,NaN,6023,Humboldt,no_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Wed13Dec17
3514,6037990100,0,NaN,NaN,NaN,6037,Los Angeles,no_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Wed13Dec17
3515,6037990200,0,NaN,NaN,NaN,6037,Los Angeles,no_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Wed13Dec17
3516,6037990300,0,NaN,NaN,NaN,6037,Los Angeles,no_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Wed13Dec17
3595,6041990100,0,NaN,NaN,NaN,6041,Marin,no_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [334]:
# check # missing values in HPI2 score column
df_hpi['hpi2score'].isnull().sum()

264

In [335]:
# create column percentage of covid deaths over total cases 
ca_df_raw["covid_mortality_rate"] = (ca_df_raw["covid_deaths"]/ca_df_raw["covid_total"])*100
ca_df_raw.head()

,state,abbreviation,county,fips,t1_socioecon,t2_household,t3_minority,t4_housing,epidim,healthcare_sys,ccvi_score,long,lat,covid_total,covid_deaths,todays_date,covid_confirmed,covid_suspected,covid_total_hospitalized,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds,population,covid_cases_per_100k,covid_deaths_per_100k,covid_mortality_rate
186,California,CA,Alameda,6001,0.184,0.028,0.971,0.604,0.093,0.022,0.097,-121.892927,37.646294,78224,1060,2021-02-05,294,34,328,2498,85,2,73,1680480,4654.860516,63.077216,1.355083
187,California,CA,Alpine,6003,0.813,0.787,0.747,0.989,0.288,0.003,0.515,-119.822359,38.596786,72,0,2021-02-05,None,None,None,None,None,None,None,1209,5955.334988,0.000000,0.000000
188,California,CA,Amador,6005,0.387,0.303,0.612,0.593,0.396,0.006,0.156,-120.656960,38.445831,3288,35,2021-02-05,7,1,8,52,2,0,2,40446,8129.357662,86.535133,1.064477
189,California,CA,Butte,6007,0.607,0.327,0.776,0.758,0.057,0.111,0.235,-121.600525,39.667278,10386,147,2021-02-05,31,0,31,451,8,0,9,196880,5275.294596,74.664770,1.415367
190,California,CA,Calaveras,6009,0.362,0.639,0.548,0.126,0.230,0.032,0.098,-120.552913,38.205371,1808,23,2021-02-05,5,0,5,33,1,0,5,46319,3903.365789,49.655649,1.272124


In [336]:
# create column percentage of hospital beds available over confirmed cases.
ca_df_raw["covid_mortality_rate"] = (ca_df_raw["covid_deaths"]/ca_df_raw["covid_total"])*100
ca_df_raw.head()

,state,abbreviation,county,fips,t1_socioecon,t2_household,t3_minority,t4_housing,epidim,healthcare_sys,ccvi_score,long,lat,covid_total,covid_deaths,todays_date,covid_confirmed,covid_suspected,covid_total_hospitalized,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds,population,covid_cases_per_100k,covid_deaths_per_100k,covid_mortality_rate
186,California,CA,Alameda,6001,0.184,0.028,0.971,0.604,0.093,0.022,0.097,-121.892927,37.646294,78224,1060,2021-02-05,294,34,328,2498,85,2,73,1680480,4654.860516,63.077216,1.355083
187,California,CA,Alpine,6003,0.813,0.787,0.747,0.989,0.288,0.003,0.515,-119.822359,38.596786,72,0,2021-02-05,None,None,None,None,None,None,None,1209,5955.334988,0.000000,0.000000
188,California,CA,Amador,6005,0.387,0.303,0.612,0.593,0.396,0.006,0.156,-120.656960,38.445831,3288,35,2021-02-05,7,1,8,52,2,0,2,40446,8129.357662,86.535133,1.064477
189,California,CA,Butte,6007,0.607,0.327,0.776,0.758,0.057,0.111,0.235,-121.600525,39.667278,10386,147,2021-02-05,31,0,31,451,8,0,9,196880,5275.294596,74.664770,1.415367
190,California,CA,Calaveras,6009,0.362,0.639,0.548,0.126,0.230,0.032,0.098,-120.552913,38.205371,1808,23,2021-02-05,5,0,5,33,1,0,5,46319,3903.365789,49.655649,1.272124


In [450]:
# Create a TSA dataframe for total cases
tsa_total_cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
tsa_total_cases = tsa_total_cases.loc[df_covid_cc_US['Province_State'] == "California"]

# Grab the columns that we want
tsa_total_cases_clean = tsa_total_cases.iloc[:,10:-1]

# Clean Combined_Key Column
tsa_total_cases_clean['Combined_Key'] =  tsa_total_cases_clean['Combined_Key'].apply(lambda x: x.split(",")[0])

# extract rows 'Out of CA', 'Unassigned'
tsa_total_cases_clean.drop(tsa_total_cases_clean.loc[tsa_total_cases_clean['Combined_Key']=="Out of CA"].index, inplace=True)
tsa_total_cases_clean.drop(tsa_total_cases_clean.loc[tsa_total_cases_clean['Combined_Key']=="Unassigned"].index, inplace=True)

tsa_total_cases_clean.rename(columns={"Combined_Key":"Date"}, inplace = True)

tsa_total_cases_clean = tsa_total_cases_clean.set_index("Date")
tsa_total_cases_clean = tsa_total_cases_clean.transpose()

# Change the index type to datetime
#tsa_total_cases_clean.index = pd.to_datetime(tsa_total_cases_clean.index)

# Add new date
tsa_total_cases_clean["New Date"] = tsa_total_cases_clean.index

tsa_total_cases_clean.tail()

Date,Alameda,Alpine,Amador,Butte,Calaveras,Colusa,Contra Costa,Del Norte,El Dorado,Fresno,Glenn,Humboldt,Imperial,Inyo,Kern,Kings,Lake,Lassen,Los Angeles,Madera,Marin,Mariposa,Mendocino,Merced,Modoc,Mono,Monterey,Napa,Nevada,Orange,Placer,Plumas,Riverside,Sacramento,San Benito,San Bernardino,San Diego,San Francisco,San Joaquin,San Luis Obispo,San Mateo,Santa Barbara,Santa Clara,Santa Cruz,Shasta,Sierra,Siskiyou,Solano,Sonoma,Stanislaus,Sutter,Tehama,Trinity,Tulare,Tuolumne,Ventura,Yolo,Yuba,New Date
1/31/21,73771,74,3199,10177,1763,1935,57337,883,8359,88476,2074,2754,26160,1083,94113,20773,2809,5393,1116948,14443,12346,367,3434,26230,407,1165,39425,8342,3499,245978,18430,626,271910,86388,5401,274429,238042,31427,61901,17605,35466,28567,101964,13383,10333,95,1532,27706,26108,50983,8264,4660,307,44518,3549,69931,11698,5324,1/31/21
2/1/21,74087,74,3227,10215,1778,1952,57580,884,8443,88852,2083,2815,26206,1087,94467,20812,2826,5413,1121107,14496,12372,368,3453,26674,411,1166,39425,8404,3528,247035,18608,626,275872,86560,5425,275076,239142,31563,62146,17887,35882,28830,102427,13606,10438,95,1558,28191,26189,51167,8355,4660,308,45243,3594,70799,11771,5389,2/1/21
2/2/21,74661,76,3234,10263,1785,1972,57812,886,8479,89272,2086,2836,26224,1091,94994,20852,2843,5413,1124975,14572,12408,371,3474,26801,419,1169,39966,8449,3540,247886,18638,627,276931,86856,5432,275372,240050,31703,62737,18038,36052,29009,102836,13648,10534,95,1576,28290,26250,51416,8355,4781,308,45398,3615,71212,11824,5389,2/2/21
2/3/21,74959,78,3275,10307,1786,1989,58021,888,8552,89542,2090,2849,26317,1108,95554,20894,2857,5421,1129730,14595,12448,373,3487,26914,421,1176,40104,8505,3577,248510,18670,630,277724,87107,5444,275960,241018,31788,62890,18186,36214,29191,103236,13683,10524,95,1580,28410,26306,51741,8428,4781,308,45541,3633,71557,11886,5423,2/3/21
2/4/21,75175,78,3281,10355,1799,1999,58248,892,8622,89748,2098,2877,26363,1117,95993,20981,2869,5435,1134595,14605,12520,374,3513,27018,424,1177,40237,8534,3598,249195,18738,631,279189,87415,5463,276606,242616,31983,63258,18353,36451,29431,103748,13790,10573,95,1593,28511,26430,51998,8474,4781,308,45647,3649,72124,11933,5424,2/4/21


In [451]:
# Create a TSA dataframe for total deaths
tsa_total_death = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
tsa_total_death = tsa_total_death.loc[df_covid_deaths_US['Province_State'] == "California"]

# Grab the columns that we want
tsa_total_death_clean = tsa_total_death.iloc[:,10:-1]
tsa_total_death_clean.drop(columns = ["Population"],inplace = True, axis=1)

# Clean Combined_Key Column
tsa_total_death_clean['Combined_Key'] =  tsa_total_death_clean['Combined_Key'].apply(lambda x: x.split(",")[0])

# # extract rows 'Out of CA', 'Unassigned'
tsa_total_death_clean.drop(tsa_total_death_clean.loc[tsa_total_death_clean['Combined_Key']=="Out of CA"].index, inplace=True)
tsa_total_death_clean.drop(tsa_total_death_clean.loc[tsa_total_death_clean['Combined_Key']=="Unassigned"].index, inplace=True)

tsa_total_death_clean.rename(columns={"Combined_Key":"Date"}, inplace = True)

tsa_total_death_clean = tsa_total_death_clean.set_index("Date")
tsa_total_death_clean = tsa_total_death_clean.transpose()

# Change the index type to datetime
#tsa_total_cases_clean.index = pd.to_datetime(tsa_total_cases_clean.index)

# Add new date
tsa_total_death_clean["New Date"] = tsa_total_death_clean.index

tsa_total_death_clean.tail()

Date,Alameda,Alpine,Amador,Butte,Calaveras,Colusa,Contra Costa,Del Norte,El Dorado,Fresno,Glenn,Humboldt,Imperial,Inyo,Kern,Kings,Lake,Lassen,Los Angeles,Madera,Marin,Mariposa,Mendocino,Merced,Modoc,Mono,Monterey,Napa,Nevada,Orange,Placer,Plumas,Riverside,Sacramento,San Benito,San Bernardino,San Diego,San Francisco,San Joaquin,San Luis Obispo,San Mateo,Santa Barbara,Santa Clara,Santa Cruz,Shasta,Sierra,Siskiyou,Solano,Sonoma,Stanislaus,Sutter,Tehama,Trinity,Tulare,Tuolumne,Ventura,Yolo,Yuba,New Date
1/31/21,980,0,33,138,23,10,529,3,79,1137,23,30,532,29,624,174,32,16,16770,175,172,5,38,350,4,4,285,54,74,3062,199,6,3091,1239,52,1839,2619,324,853,168,382,291,1414,148,142,0,13,122,260,828,86,47,5,588,49,598,149,31,1/31/21
2/1/21,982,0,34,142,23,10,529,3,81,1173,23,30,549,29,624,177,33,17,16857,175,232,5,38,354,4,4,285,54,74,3109,201,6,3171,1262,54,1856,2619,324,870,177,382,298,1418,148,145,0,13,122,265,834,89,47,5,598,49,618,149,32,2/1/21
2/2/21,991,0,34,142,23,11,531,3,81,1185,23,31,551,29,626,177,33,17,17077,184,174,5,39,357,4,4,285,55,74,3162,201,6,3251,1271,54,1917,2629,324,871,178,395,303,1433,149,150,0,13,127,267,842,89,47,5,606,49,629,156,32,2/2/21
2/3/21,999,0,34,145,23,11,532,3,83,1198,23,31,551,29,641,177,33,17,17324,184,174,5,39,359,4,4,286,57,74,3199,205,6,3278,1281,54,1966,2683,328,874,187,395,316,1473,153,153,0,13,129,267,848,91,47,5,619,50,634,158,32,2/3/21
2/4/21,1021,0,34,145,23,11,542,3,85,1224,23,31,551,29,650,189,33,18,17555,184,174,5,39,359,4,4,286,58,74,3249,206,6,3309,1288,55,2041,2738,328,878,192,395,330,1505,155,157,0,13,134,270,853,91,47,5,632,51,648,158,32,2/4/21


### Data Visualizations

In [452]:
# TSA Total Cases

import plotly.graph_objects as go
df = tsa_total_cases_clean

# plotly figure setup
fig = go.Figure()

# one trace for each df column
fig.add_trace(go.Scatter(name="Selection 1",x=df["New Date"].values, y=df["Alameda"].values))

# one trace for each df column
fig.add_trace(go.Scatter(name="Selection 2",x=df["New Date"].values, y=df["Alameda"].values))


# one button for each df column
updatemenu= []
buttons=[]

# add second buttons
buttons2 = []

for i in list(df.columns[:-1]):
  buttons.append(dict(method='restyle',label = str(i),args=[{'x':[df["New Date"].values],'y':[df[i].values]},[0]])
                  )
  
for i in list(df.columns[:-1]):
  buttons2.append(dict(method='restyle',label = str(i),args=[{'x':[df["New Date"].values],'y':[df[i].values]},[1]])
                  )
  
# some adjustments to the updatemenus
button_layer_1_height = 1.25
updatemenu = list([dict(
    buttons = buttons,
    direction="down",
    pad={"r":10,"t":10},
    showactive=True,
    x= 0.1,
    xanchor="left",
    y= button_layer_1_height,
    yanchor="top",
    font = dict(color = "blue")
),dict(
    buttons = buttons2,
    direction="down",
    pad={"r":10,"t":10},
    showactive=True,
    x= 0.37,
    xanchor="left",
    y=button_layer_1_height,
    yanchor="top",  font = dict(color = "red"))])

fig.update_layout(showlegend=True, updatemenus=updatemenu, title_text = "Covid19 Cases")
fig.update_xaxes(categoryorder= 'array', categoryarray= df.index)
fig.show()

In [453]:
# TSA Total Death

import plotly.graph_objects as go
df1 = tsa_total_death_clean

# plotly figure setup
fig = go.Figure()

# one trace for each df column
fig.add_trace(go.Scatter(name="Selection 1",x=df1["New Date"].values, y=df1["Alameda"].values))

# one trace for each df column
fig.add_trace(go.Scatter(name="Selection 2",x=df1["New Date"].values, y=df1["Alameda"].values))


# one button for each df column
updatemenu= []
buttons=[]

# add second buttons
buttons2 = []

for i in list(df.columns[:-1]):
  buttons.append(dict(method='restyle',label = str(i),args=[{'x':[df1["New Date"].values],'y':[df1[i].values]},[0]])
                  )
  
for i in list(df.columns[:-1]):
  buttons2.append(dict(method='restyle',label = str(i),args=[{'x':[df1["New Date"].values],'y':[df1[i].values]},[1]])
                  )
  
# some adjustments to the updatemenus
button_layer_1_height = 1.25
updatemenu = list([dict(
    buttons = buttons,
    direction="down",
    pad={"r":10,"t":10},
    showactive=True,
    x= 0.1,
    xanchor="left",
    y= button_layer_1_height,
    yanchor="top",
    font = dict(color = "blue")
),dict(
    buttons = buttons2,
    direction="down",
    pad={"r":10,"t":10},
    showactive=True,
    x= 0.37,
    xanchor="left",
    y=button_layer_1_height,
    yanchor="top",  font = dict(color = "red"))])

fig.update_layout(showlegend=True, updatemenus=updatemenu, title_text = "Covid19 Deaths")
fig.update_xaxes(categoryorder= 'array', categoryarray= df1.index)
fig.show()

In [457]:
import plotly.graph_objects as go
df = ca_df_raw
date_of_data = str(ca_df_raw.iloc[0,15].date())

# Drop rows that are not relevant for the plot.
df = df.drop(['ccvi_score','state', 'abbreviation','fips', 't1_socioecon',
       't2_household', 't3_minority', 't4_housing', 'epidim', 'healthcare_sys',
       'long', 'lat', 'population',"todays_date" ],axis=1)
# plotly figure setup
fig = go.Figure()

# one trace for each df column
fig.add_trace(go.Bar(name="Selection 1",x=df['county'].values, y=df["covid_total"].values))

# one trace for each df column
fig.add_trace(go.Bar(name="Selection 2",x=df['county'].values, y=df["covid_total"].values))


# one button for each df column
updatemenu= []
buttons=[]

# add second buttons
buttons2 = []

for i in list(df.columns):
  buttons.append(dict(method='restyle',label = str(i),args=[{'x':[df["county"].values],'y':[df[i].values]},[0]])
                  )
  
for i in list(df.columns):
  buttons2.append(dict(method='restyle',label = str(i),args=[{'x':[df["county"].values],'y':[df[i].values]},[1]])
                  )

# some adjustments to the updatemenus
button_layer_1_height = 1.23
updatemenu = list([dict(
    buttons = buttons,
    direction="down",
    pad={"r":10,"t":10},
    showactive=True,
    x= 0.1,
    xanchor="left",
    y= button_layer_1_height,
    yanchor="top",
    font = dict(color = "blue")
),dict(
    buttons = buttons2,
    direction="down",
    pad={"r":10,"t":10},
    showactive=True,
    x= 0.37,
    xanchor="left",
    y=button_layer_1_height,
    yanchor="top",font = dict(color = "red"))])


fig.update_layout(showlegend=True, updatemenus=updatemenu, title_text = date_of_data)
fig.show()

In [340]:
import folium
import copy
from folium.plugins import MarkerCluster

# We take a copy of the original data frame
df_geo = copy.deepcopy(ca_df_raw)

# swap columns long and lat to lat and long.
columns_titles = ['state', 'abbreviation', 'county', 'fips', 't1_socioecon',
       't2_household', 't3_minority', 't4_housing', 'epidim', 'healthcare_sys',
       'ccvi_score', 'lat','long', 'covid_total', 'covid_deaths',
       'todays_date', 'covid_confirmed', 'covid_suspected',
       'covid_total_hospitalized', 'all_hospital_beds',
       'icu_covid_confirmed_patients', 'icu_suspected_covid_patients',
       'icu_available_beds', 'population', 'covid_cases_per_100k',
       'covid_deaths_per_100k']

df_geo = df_geo.reindex(columns=columns_titles)

df_geo.head()

# We create a list for each row in the dataframe.
geo_lst = df_geo.values.tolist()

# We create an empty list.
long_lat_lst = []

#We loop through the list and only append the long and lat values.
for i in geo_lst:
    long_lat_lst.append((i[12],i[11]))

# we draw the folium map
map = folium.Map(location=[36.17, -119.7462],zoom_start = 6)


for j in range(len(long_lat_lst)):
        
        # We grab the infos that we want to add to the popup.
        county_name = df_geo.iloc[j, df_geo.columns.get_loc("county")]
        ccvi_score = df_geo.iloc[j, df_geo.columns.get_loc("ccvi_score")]
        covid_cases_per_100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_cases_per_100k")]
        covid_deaths_per_100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_deaths_per_100k")]
        covid_deaths_per100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_deaths")]
        popup_text = "County: {county_name} <br>""CCVI Score: {ccvi_score} <br>""Covid Total Per 100K: {covid_cases_per_100k} <br>""Covid Deaths Per 100K: {covid_deaths_per_100k} <br>".format(county_name=county_name,ccvi_score=ccvi_score, covid_cases_per_100k =covid_cases_per_100k, covid_deaths_per_100k = covid_deaths_per_100k)

        # We create the circles in the map. 
        folium.Circle(
        radius=df.iloc[j]['covid_deaths_per_100k'] * 200,
        weight=1,  # thickness of the border
        color='red',  # this is the color of the border
        opacity=0.3,  # this is the alpha for the border
        fill_color='red',  # fill is inside the circle
        fill_opacity=0.1,  # we will make that less opaque so we can see layers
        location = [long_lat_lst[j][1],long_lat_lst[j][0]],
        popup = folium.Popup(popup_text,max_width=300),
        fill = True).add_to(map)

map

In [341]:
# chloropleth map prep

# check the data types
df_geo.dtypes

state                                   object
abbreviation                            object
county                                  object
fips                                     int64
t1_socioecon                           float64
t2_household                           float64
t3_minority                            float64
t4_housing                             float64
epidim                                 float64
healthcare_sys                         float64
ccvi_score                             float64
lat                                    float64
long                                   float64
covid_total                              int64
covid_deaths                             int64
todays_date                     datetime64[ns]
covid_confirmed                         object
covid_suspected                         object
covid_total_hospitalized                object
all_hospital_beds                       object
icu_covid_confirmed_patients            object
icu_suspected

We have objects that contain numeric data:
- covid_confirmed
- covid_suspected
- covid_total_hospitalized
- all_hospital_beds
- icu_covid_confirmed_patients
- icu_suspected_covid_patients
- icu_available_beds

In [342]:
numerical_cols = ['covid_confirmed', 'covid_suspected',
'covid_total_hospitalized', 'all_hospital_beds', 'icu_covid_confirmed_patients',
'icu_suspected_covid_patients', 'icu_available_beds']

# convert these to floats using the pandas.Series.astype function
for column in numerical_cols:
    df_geo[column] = df_geo[column].astype(float)

In [343]:
# check what directory we're in
!ls

sample_data


Folium is a python library that allows you to create multiple types of Leaflet maps. To create a choropleth map, Folium requires a Geo JSON file, including geospatial data of the region. For a choropleth map of CA counties, we need a Geo JSON file that defines the boundaries of the counties. You can download the CA counties file from this link: https://github.com/codeforamerica/click_that_hood/blob/master/public/data/california-counties.geojson

To bind the data frame & json file, the name of the county in our data frame must exactly match the name in the json file. Let's check the names in the json file.

In [344]:
import json
# path to geojson map data for CA counties
CA_COUNTIES_GEO_FILEPATH = 'https://storage.googleapis.com/ca_county_map/ca-counties.geojson.txt' #'sample_data/ca-counties.geojson.txt' 

# open json file. json.load() methods returns a python dictionary
with open(CA_COUNTIES_GEO_FILEPATH) as counties_file:
    counties_json = json.load(counties_file)
    
# loop through dictionary to obtain name of counties in the json file
names_json = []
for index in range(len(counties_json['features'])):
    names_json.append(counties_json['features'][index]['properties']['name'])
    
names_json
sorted(names_json)

# remove quotes
for name in names_json:
    name = name.replace("'","")

FileNotFoundError: ignored

In [ ]:
df_geo.county

In [ ]:
# make a list of names in the data frame
df_names = df_geo.county.tolist()

# names in the json file - the same order as in the data frame 
geojson_names = ['Alameda',
 'Alpine',
 'Amador',
 'Butte',
 'Calaveras',
 'Colusa',
 'Contra Costa',
 'Del Norte',
 'El Dorado',
 'Fresno',
 'Glenn',
 'Humboldt',
 'Imperial',
 'Inyo',
 'Kern',
 'Kings',
 'Lake',
 'Lassen',
 'Los Angeles',
 'Madera',
 'Marin',
 'Mariposa',
 'Mendocino',
 'Merced',
 'Modoc',
 'Mono',
 'Monterey',
 'Napa',
 'Nevada',
 'Orange',
 'Placer',
 'Plumas',
 'Riverside',
 'Sacramento',
 'San Benito',
 'San Bernardino',
 'San Diego',
 'San Francisco',
 'San Joaquin',
 'San Luis Obispo',
 'San Mateo',
 'Santa Barbara',
 'Santa Clara',
 'Santa Cruz',
 'Shasta',
 'Sierra',
 'Siskiyou',
 'Solano',
 'Sonoma',
 'Stanislaus',
 'Sutter',
 'Tehama',
 'Trinity',
 'Tulare',
 'Tuolumne',
 'Ventura',
 'Yolo',
 'Yuba']

# replace data frame names by json names
df_geo.replace(dict(zip(df_names, geojson_names)), inplace=True)
df_geo.county

In [ ]:
# draw the folium map for covid death rates in CA counties
counties_map = folium.Map(location=[36.17, -119.7462],zoom_start = 6, tiles = 'Stamen Terrain')#'Stamen Toner')

folium.Choropleth(
  geo_data = CA_COUNTIES_GEO_FILEPATH,
  data = df_geo,
  columns = ['county', 'covid_deaths_per_100k'],
  key_on = 'feature.properties.name',
  fill_color = 'RdPu', 
  fill_opacity = 1, 
  line_opacity = 1,
  legend_name='CA County Covid Deaths per 100k Population',
  smooth_factor = 0).add_to(counties_map)

for j in range(len(long_lat_lst)):
        
        # We grab the infos that we want to add to the popup.
        county_name = df_geo.iloc[j, df_geo.columns.get_loc("county")]
        ccvi_score = df_geo.iloc[j, df_geo.columns.get_loc("ccvi_score")]
        covid_cases_per_100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_cases_per_100k")]
        covid_deaths_per_100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_deaths_per_100k")]
        covid_deaths_per100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_deaths")]
        popup_text = "County: {county_name} <br>""CCVI Score: {ccvi_score} <br>""Covid Total Per 100K: {covid_cases_per_100k} <br>""Covid Deaths Per 100K: {covid_deaths_per_100k} <br>".format(county_name=county_name,ccvi_score=ccvi_score, covid_cases_per_100k =covid_cases_per_100k, covid_deaths_per_100k = covid_deaths_per_100k)
"""
        #We create the circles in the map. 
        folium.Circle(
        radius=100,
        weight=2,  
        color='black',  
        location = [long_lat_lst[j][1],long_lat_lst[j][0]],
        popup = folium.Popup(popup_text,max_width=300),
        fill = True).add_to(counties_map)
"""
        

#display map
counties_map

In [ ]:
# draw the folium map for CCVI vulnerability rates in CA counties
counties_map = folium.Map(location=[36.17, -119.7462],zoom_start = 6, tiles ='Stamen Terrain') # 'Stamen Toner')

folium.Choropleth(
  geo_data = CA_COUNTIES_GEO_FILEPATH,
  data = df_geo,
  columns = ['county', 'ccvi_score'],
  key_on = 'feature.properties.name',
  fill_color = 'RdPu', 
  fill_opacity = 1, 
  line_opacity = 1,
  legend_name='CA County Covid Vulnerability (CCVI score)',
  smooth_factor = 0).add_to(counties_map)

for j in range(len(long_lat_lst)):
        
        # Grab info to add to the popup.
        county_name = df_geo.iloc[j, df_geo.columns.get_loc("county")]
        ccvi_score = df_geo.iloc[j, df_geo.columns.get_loc("ccvi_score")]
        covid_cases_per_100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_cases_per_100k")]
        covid_deaths_per_100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_deaths_per_100k")]
        covid_deaths_per100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_deaths")]
        popup_text = "County: {county_name} <br>""CCVI Score: {ccvi_score} <br>""Covid Total Per 100K: {covid_cases_per_100k} <br>""Covid Deaths Per 100K: {covid_deaths_per_100k} <br>".format(county_name=county_name,ccvi_score=ccvi_score, covid_cases_per_100k =covid_cases_per_100k, covid_deaths_per_100k = covid_deaths_per_100k)
"""
        # Create circles in the map. 
        folium.Circle(
        radius=100,
        weight=2,  
        color='black',  
        location = [long_lat_lst[j][1],long_lat_lst[j][0]],
        popup = folium.Popup(popup_text,max_width=300),
        fill = True).add_to(counties_map)
"""
        

#display map
counties_map

In [ ]:
# draw the folium map for Covid Case rates in CA counties
counties_map = folium.Map(location=[36.17, -119.7462],zoom_start = 6, tiles ='Stamen Terrain') # 'Stamen Toner')

folium.Choropleth(
  geo_data = CA_COUNTIES_GEO_FILEPATH,
  data = df_geo,
  columns = ['county', 'covid_cases_per_100k'],
  key_on = 'feature.properties.name',
  fill_color = 'RdPu', 
  fill_opacity = 1, 
  line_opacity = 1,
  legend_name='CA County Covid Cases per 100k',
  smooth_factor = 0).add_to(counties_map)

for j in range(len(long_lat_lst)):
        
        # Grab info to add to the popup.
        county_name = df_geo.iloc[j, df_geo.columns.get_loc("county")]
        ccvi_score = df_geo.iloc[j, df_geo.columns.get_loc("ccvi_score")]
        covid_cases_per_100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_cases_per_100k")]
        covid_deaths_per_100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_deaths_per_100k")]
        covid_deaths_per100k = df_geo.iloc[j, df_geo.columns.get_loc("covid_deaths")]
        popup_text = "County: {county_name} <br>""CCVI Score: {ccvi_score} <br>""Covid Total Per 100K: {covid_cases_per_100k} <br>""Covid Deaths Per 100K: {covid_deaths_per_100k} <br>".format(county_name=county_name,ccvi_score=ccvi_score, covid_cases_per_100k =covid_cases_per_100k, covid_deaths_per_100k = covid_deaths_per_100k)

        # Create circles in the map. 
        folium.Circle(
        radius=100,
        weight=2,  
        color='black',  
        location = [long_lat_lst[j][1],long_lat_lst[j][0]],
        popup = folium.Popup(popup_text,max_width=300),
        fill = True).add_to(counties_map)

        

#display map
counties_map

In [ ]:
import seaborn as sns # for visuals
import matplotlib.pyplot as plt # for graphs
# how graphs are printed
%matplotlib inline 
plt.style.use('ggplot')

# Plot Covid Vulnerability by County

# order plot by ccvi score instead of default alphabetical county name
plot_order = ca_df_raw.sort_values(by='ccvi_score', ascending=False).county.values

# plot
ax = sns.barplot(x='ccvi_score', y='county', data=ca_df_raw, order=plot_order)#, palette='RdYlGn')
fig = plt.gcf()
fig.set_size_inches(5, 10)
plt.xlabel("Covid Vulnerability (CCVI Score)")
plt.ylabel("County")
plt.title("Covid Vulnerability by County in CA")

# plt.savefig("ccvi_by_county.png", format='png',dpi=150)

In [ ]:
# Plot Covid Total Deaths by County
plot_order = ca_df_raw.sort_values(by='covid_total', ascending=False).county.values
ax = sns.barplot(x='covid_total', y='county', data=ca_df_raw, order=plot_order, palette='RdYlGn')
fig = plt.gcf()
fig.set_size_inches(5, 10)
plt.xlabel("Covid Total Deaths")
plt.ylabel("County")
plt.title("Covid Total Deaths by County in CA")

In [ ]:
# Plot Covid Total Deaths by County
plot_order = ca_df_raw.sort_values(by='covid_deaths_per_100k', ascending=False).county.values
ax = sns.barplot(x='covid_deaths_per_100k', y='county', data=ca_df_raw, order=plot_order, palette='RdYlGn')
fig = plt.gcf()
fig.set_size_inches(5, 10)
plt.xlabel("Covid Deaths by Population (100k)")
plt.ylabel("County")
plt.title("Covid Deaths in CA Normalized by Population")